# 利用相似性搜索实现中文查询

## 准备

In [1]:
!pip install faiss-cpu
!pip install sentence-transformers
!pip install pandas

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 创建知识库

In [1]:
import pandas as pd
data = [['《西游记》是中国古典小说四大名著之一，又称《大唐西游记》', '小说'],
['牛顿的主要著作是《数学原理》。这部著作于1687年首次出版，被认为是自然科学史上最重要的著作之一。', '数学'],
['东京迪斯尼乐园（Tokyo Disneyland）和东京迪士尼海洋（Tokyo DisneySea）是东京都内的两座迪士尼度假区。', '旅游'],
['电视连续剧《西游记》（1986年版）的主题曲是《云宫迅音》。', '音乐']]
df = pd.DataFrame(data, columns = ['文本', '分类'])

df

,文本,分类
0,《西游记》是中国古典小说四大名著之一，又称《大唐西游记》,小说
1,牛顿的主要著作是《数学原理》。这部著作于1687年首次出版，被认为是自然科学史上最重要的著作之一。,数学
2,东京迪斯尼乐园（Tokyo Disneyland）和东京迪士尼海洋（Tokyo Disney...,旅游
3,电视连续剧《西游记》（1986年版）的主题曲是《云宫迅音》。,音乐


## 创建向量

In [2]:
%%time

from sentence_transformers import SentenceTransformer
text = df['文本']
encoder = SentenceTransformer("/models/bge-large-zh-v1.5")
vectors = encoder.encode(text)

CPU times: user 3.42 s, sys: 907 ms, total: 4.33 s
Wall time: 4.05 s


## 创建 Faiss 索引

In [3]:
%%time

import faiss

vector_dimension = vectors.shape[1]
index = faiss.IndexFlatL2(vector_dimension)
faiss.normalize_L2(vectors)
index.add(vectors)

CPU times: user 15.5 ms, sys: 3.87 ms, total: 19.4 ms
Wall time: 21.5 ms


## 创建搜索向量

In [4]:
%%time

import numpy as np

search_text = '能介绍下西游记这个小说么?'
search_vector = encoder.encode(search_text)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

CPU times: user 34.6 ms, sys: 0 ns, total: 34.6 ms
Wall time: 35 ms


## 从索引中查找所有最近相邻

In [5]:
%%time

k = index.ntotal
distances, ann = index.search(_vector, k=k)

CPU times: user 49 µs, sys: 0 ns, total: 49 µs
Wall time: 53.4 µs


## 结果排序

In [6]:
%%time

results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})
results

CPU times: user 341 µs, sys: 67 µs, total: 408 µs
Wall time: 378 µs


,distances,ann
0,0.654768,0
1,1.141115,3
2,1.452280,1
3,1.587813,2


## 合并表格数据

In [7]:
%%time

merge=pd.merge(results,df,left_on='ann',right_index=True)
merge

CPU times: user 3.62 ms, sys: 0 ns, total: 3.62 ms
Wall time: 5.9 ms


,distances,ann,文本,分类
0,0.654768,0,《西游记》是中国古典小说四大名著之一，又称《大唐西游记》,小说
1,1.141115,3,电视连续剧《西游记》（1986年版）的主题曲是《云宫迅音》。,音乐
2,1.452280,1,牛顿的主要著作是《数学原理》。这部著作于1687年首次出版，被认为是自然科学史上最重要的著作之一。,数学
3,1.587813,2,东京迪斯尼乐园（Tokyo Disneyland）和东京迪士尼海洋（Tokyo Disney...,旅游


## 再问个问题

In [8]:
search_text = '孙悟空是哪部小说中的人物?'
search_vector = encoder.encode(search_text)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

k = index.ntotal
distances, ann = index.search(_vector, k=k)

results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})
merge=pd.merge(results,df,left_on='ann',right_index=True)

merge

,distances,ann,文本,分类
0,0.850697,0,《西游记》是中国古典小说四大名著之一，又称《大唐西游记》,小说
1,1.285335,3,电视连续剧《西游记》（1986年版）的主题曲是《云宫迅音》。,音乐
2,1.432754,1,牛顿的主要著作是《数学原理》。这部著作于1687年首次出版，被认为是自然科学史上最重要的著作之一。,数学
3,1.646388,2,东京迪斯尼乐园（Tokyo Disneyland）和东京迪士尼海洋（Tokyo Disney...,旅游
